In [1]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers import Input
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
#from sklearn.model_selection import cross_val_score
#from sklearn.cross_validation import cross_val_score
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from sklearn import metrics

from sklearn.metrics import confusion_matrix

import matplotlib.pyplot as plt

import io
import os

from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

from keras import regularizers
import time
import time

from keras import regularizers
from scipy import stats
import seaborn as sns
import pickle
from keras.callbacks import ModelCheckpoint, TensorBoard, EarlyStopping, ReduceLROnPlateau

from keras.models import Model, load_model

from sklearn.model_selection import train_test_split


from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler

/home/kodzo.apedo/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/home/kodzo.apedo/anaconda3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
#chargement des donnees 

df = pd.read_csv('/home/kodzo.apedo/Bureau/Visu/mesdonnees/mesdonnees.csv')

In [3]:
ds = df.values

In [4]:
RANDOM_SEED = 42 #used to help randomly select the data points
TEST_PCT = 0.30 

In [5]:
X = ds[:,0:34].astype(float)
Y = ds[:,34]

In [6]:
#Encodons la classe "Classe"

encoder = LabelEncoder()
encoder.fit(Y)
encoder_Y = encoder.transform(Y)

In [7]:
encoder_Y

array([ 1,  1,  1, ..., 19, 19, 19])

In [8]:
# Redefinissons nos classe

dummy_y = np_utils.to_categorical(encoder_Y)

In [9]:
Yd = dummy_y

In [10]:
X_train, X_test, Yd_train, Yd_test = train_test_split(X, Yd, test_size = TEST_PCT, random_state = RANDOM_SEED)

In [11]:
nb_class = 20 # Nombre de classe

nb_epoch = 10

batch_size = 1000

#Couche d'entrée

input_dim = X_train.shape[1] #numbre de colonne, 

encoding_dim = 500 # Dimension d'encodage
nb_class = 20 # Nombre de classe
hidden_dim = encoding_dim - 200 #i.e. 7

hidden_dim2 = hidden_dim - 200


learning_rate = 1e-7

In [12]:

#Debut du decompte du temps
start_time = time.time()

input_layer = Input(shape=(input_dim, ))

encoder = Dense(encoding_dim, activation='tanh', 
                activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation='tanh')(encoder)
#encoder = Dense(hidden_dim2, activation='tanh')(encoder)
encoder = Dense(input_dim, activation='relu')(encoder)


print("Temps d execution : %s secondes ---" %(time.time() - start_time))


Temps d execution : 0.1270599365234375 secondes ---


In [13]:

autoencoder = Model(inputs=input_layer, outputs=encoder)


autoencoder.summary()

autoencoder.compile(metrics=['accuracy'],
                    loss='mse',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="11ENCODER-FOR-PRED-4CP.h5",
                               save_best_only=True,
                               verbose=0)



history = autoencoder.fit(X_train, X_train,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(X_test, X_test), callbacks = [cp]).history

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 34)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 500)               17500     
_________________________________________________________________
dense_2 (Dense)              (None, 300)               150300    
_________________________________________________________________
dense_3 (Dense)              (None, 34)                10234     
Total params: 178,034
Trainable params: 178,034
Non-trainable params: 0
_________________________________________________________________
Train on 279986 samples, validate on 119995 samples
Epoch 1/10
279986/279986 [==============================] - 7s 26us/step - loss: 0.3238 - acc: 0.8094 - val_loss: 0.1969 - val_acc: 0.9196
Epoch 2/10
148000/279986 [==============>...............] - ETA: 2s - loss: 0

KeyboardInterrupt: 

In [ ]:
plt.plot(history['loss'], linewidth=2, label='Train')
plt.plot(history['val_loss'], linewidth=2, label='Test')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.plot(history['acc'], linewidth=2, label='Train')
plt.plot(history['val_acc'], linewidth=2, label='Test')
plt.legend(loc='lower right')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
#Debut du decompte du temps
start_time = time.time()

model = Sequential()

model.add(autoencoder)
#model.add(Dropout(0.5))
model.add(Dense(500, activation ='relu'))
#model.add(Dropout(0.1))
model.add(Dense(300, activation ='tanh'))
#model.add(Dropout(0.1))
model.add(Dense(100, activation ='relu'))
model.add(Dropout(0.1))
model.add(Dense(nb_class, activation ='sigmoid'))


model.compile(metrics=['accuracy'], loss = 'mse', 
              optimizer='adam')

model.summary()


checkpointer = ModelCheckpoint(filepath='11COMB-PRED-4CP.h5', 
                               verbose=2, save_best_only=True)



history = model.fit(X_train, Yd_train,
             validation_data=(X_test, Yd_test),
             epochs= nb_epoch,
             batch_size=batch_size,
             shuffle=True,
             verbose=2, callbacks=[checkpointer]).history


score, acc = model.evaluate(X_test, y = Yd_test, 
               batch_size = batch_size, 
               verbose = 1)




print("Temps d execution : %s secondes ---" %(time.time() - start_time))

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
y_pred[y_pred>= 0.5] = 1
y_pred[y_pred<0.5] = 0
print (y_pred)

In [ ]:
plt.plot(history['loss'], linewidth=2, label='Train')
plt.plot(history['val_loss'], linewidth=2, label='Test')
plt.legend(loc='upper right')
plt.title('Model loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show()

In [ ]:
plt.plot(history['acc'], linewidth=2, label='Train')
plt.plot(history['val_acc'], linewidth=2, label='Test')
plt.legend(loc='lower right')
plt.title('Model accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
df1 = pd.read_csv('/home/kodzo.apedo/Bureau/STAGE/donneesTests.csv')

In [ ]:
model = load_model('COMB-PRED-4cp.h5')

In [ ]:
Xa = df1.values

In [ ]:
preds = model.predict(Xa)

In [ ]:
preds[preds>= 0.5] = 1
preds[preds<0.5] = 0
print(preds)
print("")